In [ ]:
# Save a copy of the model output df - dont use it, overwritten by ship_merged
ship_merged_raw = ship_merged.copy()

### ship-Processing of the Model Output
ship_merged = ship_merged.drop(['week_of_year_x','index','covid_max_growth'],axis=1)
ship_merged.rename(columns={'week_of_year_y':'week_of_year'},inplace=True)
ship_merged['common_product']=1

# Merge model output with 2019 dataset to obtain the pos_qty_ly
ship_merged = ship_merged.merge(ship_main_2019,on=['sell_id','week_of_year'],how='left',suffixes=('', '_y'))
ship_merged.rename(columns={'pos_qty':'pos_qty_ly','pos_dollar':'pos_dollar_ly'},inplace=True)
ship_merged.drop(['week_ending_date_y','state_y','year'],axis=1,inplace=True)


# Merge in prophet's forecast for common products
ship_merged = ship_merged.merge(prht_fcst_common, how='left', on=['sell_id','week_ending_date'])

# Use prophet's forecast if available
ship_merged = ship_merged.assign(pos_qty_rolling_6_week_med = \
                              np.where(~(ship_merged.pos_qty_prht_rolling_6_week_med.isnull()), 
                                       ship_merged.pos_qty_prht_rolling_6_week_med, 
                                       ship_merged.pos_qty_rolling_6_week_med))

ship_merged = ship_merged.drop(['pos_qty_prht_rolling_6_week_med'],axis=1)


# Subset the dataset on just the peak covid growth windows (3/14 & 3/21)
maximum_peak_data = ship_merged[(ship_merged['week_of_year']>=11) & (ship_merged['week_of_year']<=12)]

# For each sell id, Identify the growth peak corresponding to those two weeks
sell_maxium_ly = maximum_peak_data.loc[maximum_peak_data.groupby('sell_id')['Growth_perc_qty'].idxmax()][['sell_id','pos_qty_ly']]
sell_maxium_ly.rename(columns={'pos_qty_ly':'max_ship_ly'},inplace=True)

ship_merged = ship_merged.merge(sell_maxium_ly,on=['sell_id'],how='left')

def apply_forecast_value_positive(week,growth,qty_ly_rolling,qty_ty,qty_ly_max,z,third_m,second_m,fcst_qty,high_growth_decline):
      
    if week == latest_ship_week: # If the week is current ship week, set the forecast value to actual ship (just for viz purporse)
        return qty_ty
    
    elif week>latest_ship_week and week<second_m:
        return fcst_qty
    
    elif week>=second_m and high_growth_decline==True:
        return qty_ly_rolling
    elif week>=second_m:
        return (1 + growth) * qty_ly_rolling    
    else:
        return np.nan

def apply_forecast_value_negative(week,growth,qty_ly_rolling,qty_ty,qty_ly_max,z,third_m,second_m,fcst_qty,high_growth_decline):
      
    if week == latest_ship_week: # If the week is current ship week, set the forecast value to actual ship (just for viz purporse)
        return qty_ty
    
    elif week>latest_ship_week and week<second_m:
        return fcst_qty
    
    elif week>=second_m and week<third_m:
        return (1 + growth) * qty_ly_max    
    
    elif week>=third_m and high_growth_decline==True:
        return qty_ly_rolling
    elif week>=third_m:
        return (1+growth) * qty_ly_rolling
    else:
        return np.nan

def floor_growth(median,growth,week,z): # positive growing products
        
    if week>latest_ship_week and growth<median:
        return median
    else:
        return growth

def ceil_growth(median,growth,week,z): # Negative growing products
    
    if week>latest_ship_week and growth>median:
        return median
    else:
        return growth

ship_merged['forecast1'] = ship_merged[['median_baseline','forecast1','week_ending_date','negative_covid_impact']].apply(lambda x: floor_growth(*x) if x[3] == 0 else ceil_growth(*x),axis=1)
ship_merged['forecast_quantity'] = ship_merged[['week_ending_date','forecast1','pos_qty_rolling_6_week_med',
                                                'pos_qty_ty','max_ship_ly','negative_covid_impact',
                                                'third_milestone_date','second_milestone_date',
                                                'forecast_quantity','high_growth_decline']]\
                                    .apply(lambda x: apply_forecast_value_positive(*x) if x[5]==0  \
                                            else apply_forecast_value_negative(*x) ,axis=1)

ship_merged['week_ending_date'] = pd.to_datetime(ship_merged['week_ending_date'])
ship_merged['first_milestone_date'] = pd.to_datetime(ship_merged['first_milestone_date'])
ship_merged['second_milestone_date'] = pd.to_datetime(ship_merged['second_milestone_date'])
ship_merged['third_milestone_date'] = pd.to_datetime(ship_merged['third_milestone_date'])



def update_forecast_positive(week,forecast1,fcst_growth,pos_qty_rolling,fcst_qty,first_m,second_m,neg_covid, high_growth_decline):

    if week>latest_ship_week and week<second_m and high_growth_decline!=True: # Floor the value 
        if fcst_growth < forecast1:
            return (1+forecast1) * pos_qty_rolling
        else:
            return fcst_qty
    
    else:
        return fcst_qty

def update_forecast_negative(week,forecast1,fcst_growth,pos_qty_rolling,fcst_qty,first_m,second_m,neg_covid, high_growth_decline):

    if week>latest_ship_week and week<second_m and high_growth_decline!=True: # Ceil the value
        if fcst_growth > forecast1:
            return (1+forecast1) * pos_qty_rolling
        else:
            return fcst_qty
    
    else:
        return fcst_qty

#'week_ending_date','second_milestone_date_y','pos_qty_ly','pos_qty_rolling_6_week_med'
def find_promo_uplift(week,second_m,qty_ly,qty_rolling):
    if week>=(pd.to_datetime(max_train_date) + timedelta(days=14)):
        if (qty_ly - qty_rolling)>0:
            return qty_ly - qty_rolling
        else: 
            return 0 
    else:
        return 0